In [25]:
import pyspark, os, sys
from pyspark.sql import *
from pyspark import SparkConf,SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import types

spark=SparkSession.builder.getOrCreate()

"example_json"={[
"id": "1",
"full_name": [
{
    "first_name": "Jacob",
    "last_name": "Jack"
}
],
"role": "developer"
]}

schema = StructField("id", StringType(), True),
StructField("full_name", StructType([
        StructField("first_name", StringType(), True),
        StructField("last_name", StringType(), True)
        ]), True),
StructField("role", StringType(), True)



data=[(str(example_json))]
#df=spark.read.option("multiline", "true").option('inferSchema', False).json("example.json").show()

df=spark.createDataFrame(data=example_json,schema = schema)

SyntaxError: cannot assign to literal (4268188438.py, line 10)

In [14]:
def flatten(df):
    complex_fields=dict([(field.name, field.dataType)        #dict() is built-in function in python which is used to store key-value pairs
                        for field in df.schema.fields
                        if isinstance(field.dataType, (ArrayType, StructType))])   #Here we are checking if datatypes of fields are arrayType or structType
    
    while len(complex_fields)!=0:
        col_name=list(complex_fields.keys())[0]  #here as we have key-value pair which is column name and datatype in dict(), we are taking column name in 'col_name'
        if isinstance(complex_fields[column_name], StructType):
            expanded=[col(col_name+'.'+k).alias(col_name+'_'+k) for k in [n.name for n in complex_fields[col_name]]]    #converting field1.field2 to field1_field2
            df=df.select("*", *expanded).drop(col_name)
        elif isinstance(complex_fields[column_name], ArrayType):
            df=df.withColumn(col_name,explode_outer(col_name))
            
            complex_fields=dict([(field.name, field.dataType)
                        for field in df.schema.fields
                        if type(field.dataType)==ArrayType or type(field.dataType)==StructType])
            
    return df


In [15]:
df_flatten=flatten(df)

display(df_flatten)

AttributeError: 'NoneType' object has no attribute 'schema'